In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/medical-mnist")

print("Path to dataset files:", path)

/Users/I569354/Desktop/Praxisarbeiten/Studienarbeit_Sammlung/MedicaldeepLearning/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/I569354/Desktop/Praxisarbeiten/Studienarbeit_Sammlung/MedicaldeepLearning/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 84.8M/84.8M [00:07<00:00, 11.9MB/s]

Extracting files...


Path to dataset files: /Users/I569354/.cache/kagglehub/datasets/andrewmvd/medical-mnist/versions/1


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import medmnist
from medmnist import INFO, Evaluator

In [7]:
def download_dataset(data_flag):
  print(f"Dataset source information : MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

  download = True
  
  info = INFO[data_flag]

  DataClass = getattr(medmnist, info['python_class'])

  print('\nBreastMNIST')
  print(info['description'])

  # Downloading dataset
  train_dataset = DataClass(split='train', download=download)
  val_dataset = DataClass(split='val', download=download)
  test_dataset = DataClass(split='test', download=download)

  print('\nTraining Dataset:')
  print(train_dataset)
  # print(train_dataset.imgs.shape)
  if train_dataset.imgs.ndim == 3:
    print('Adding channel to images...')
    train_dataset.imgs = np.expand_dims(train_dataset.imgs, axis=-1)

  print('\nValidation Dataset:')
  print(val_dataset)
  # print(val_dataset.imgs.shape)
  if val_dataset.imgs.ndim == 3:
    print('Adding channel to images...')
    val_dataset.imgs = np.expand_dims(val_dataset.imgs, axis=-1)

  print('\nTesting Dataset:')
  print(test_dataset)
  # print(test_dataset.imgs.shape)
  if test_dataset.imgs.ndim == 3:
    print('Adding channel to images...')
    test_dataset.imgs = np.expand_dims(test_dataset.imgs, axis=-1)

  print('\nShapes of images:')
  print('Training: ', train_dataset.imgs.shape)
  print('Validation: ', val_dataset.imgs.shape)
  print('Testing: ', test_dataset.imgs.shape)

  return train_dataset, val_dataset, test_dataset

In [13]:
train_dataset, val_dataset, test_dataset = download_dataset('breastmnist')


Dataset source information : MedMNIST v3.0.2 @ https://github.com/MedMNIST/MedMNIST/

BreastMNIST
The BreastMNIST is based on a dataset of 780 breast ultrasound images. It is categorized into 3 classes: normal, benign, and malignant. As we use low-resolution images, we simplify the task into binary classification by combining normal and benign as positive and classifying them against malignant as negative. We split the source dataset with a ratio of 7:1:2 into training, validation and test set. The source images of 1×500×500 are resized into 1×28×28.
Using downloaded and verified file: /Users/I569354/.medmnist/breastmnist.npz
Using downloaded and verified file: /Users/I569354/.medmnist/breastmnist.npz
Using downloaded and verified file: /Users/I569354/.medmnist/breastmnist.npz

Training Dataset:
Dataset BreastMNIST of size 28 (breastmnist)
    Number of datapoints: 546
    Root location: /Users/I569354/.medmnist
    Split: train
    Task: binary-class
    Number of channels: 1
    Mean

In [18]:
class_labels = ['Malignant',
                'Normal/Benign']
# Check missing images in the dataset
if len(train_dataset.imgs) != 546:
  print("WARNING: Missing images in training dataset.")
  print(f"Found {len(train_dataset.imgs)}, should be 546.")
else:
  print("SUCCESS: No missing images in training dataset.")

if len(val_dataset.imgs) != 78:
  print("WARNING: Missing images in validation dataset.")
  print(f"Found {len(val_dataset.imgs)}, should be 78.")
else:
  print("SUCCESS: No missing images in validation dataset.")

if len(test_dataset.imgs) != 156:
  print("WARNING: Missing images in testing dataset.")
  print(f"Found {len(train_dataset.imgs)}, should be 156.")
else:
  print("SUCCESS: No missing images in test dataset.")

SUCCESS: No missing images in training dataset.
SUCCESS: No missing images in validation dataset.
SUCCESS: No missing images in test dataset.


In [19]:
# Normalising images in the dataset
# Pixel values lie between 0-255, 
# division by 255 will scale the values between 0 and 1
train_dataset.imgs = train_dataset.imgs/255.0 
val_dataset.imgs = val_dataset.imgs/255.0
test_dataset.imgs = test_dataset.imgs/255.0

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten

model = Sequential()
model.add(Flatten(input_shape=(28,28)))
model.add(Dense(128, activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(11,activation='softmax'))
model.summary()

/Users/I569354/Desktop/Praxisarbeiten/Studienarbeit_Sammlung/MedicaldeepLearning/.venv/lib/python3.9/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 11)             │           363 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 104,971 (410.04 KB)

 Trainable params: 104,971 (410.04 KB)

 Non-trainable params: 0 (0.00 B)